# Question 1

In [1]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
from sklearn.utils import check_random_state

In [10]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression

np.random.seed(6450345)

def make_data(N=1000, n_vars=10,
              n_classes=2):
    X = np.random.normal(size=(N,n_vars))
    y = np.random.choice(n_classes, N)
    
    return X, y

X,y = make_data(N=2000, n_vars=50000)

select = SelectKBest(f_regression, k=100)
X_sel = select.fit_transform(X, y)

clf = GradientBoostingClassifier()
scores = cross_val_score(clf, X_sel, y, cv=5, n_jobs=8)

print("Scores on each subset:")
print(scores)
avg = (100*np.mean(scores), 100*np.std(scores)/np.sqrt(scores.shape[0]))
print("Average score and uncertainty: (%.2f +- %.3f)%%" % avg)

Scores on each subset:
[ 0.67581047  0.69326683  0.6275      0.62406015  0.64661654]
Average score and uncertainty: (65.35 +- 1.211)%


The mistake that we are making here is that the "test" data used in the cross-validation is not independant of the training data because it has been used to compute the transformation and selection (i.e., it has been 'selected' already). To prevent this, one should separate the testing data and the training data from the beginning, and not use the testing data in the selection.

In [19]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

np.random.seed(6450345)

def make_data(N=1000, n_vars=10,
              n_classes=2):
    X = np.random.normal(size=(N,n_vars))
    y = np.random.choice(n_classes, N)
    
    return X, y

X,y = make_data(N=2000, n_vars=50000)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

select = SelectKBest(f_regression, k=100)
X_sel = select.fit_transform(X_train, y_train)

clf = GradientBoostingClassifier()
clf.fit(X_sel, y_train)
#scores = cross_val_score(clf, X_sel, y_train, cv=5, n_jobs=8)

X_sel_test = select.transform(X_test)
score = clf.score(X_sel_test, y_test)

print("Score:")
print(score)
#avg = (100*np.mean(scores), 100*np.std(scores)/np.sqrt(scores.shape[0]))
#print("Average score and uncertainty: (%.2f +- %.3f)%%" % avg)

Score:
0.511666666667


In this way, we recover an accuracy close to 0.5.